In [75]:
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.request import urlretrieve
import os
import requests
import time
from random import uniform
import pandas as pd

In [76]:
driver = webdriver.Chrome('C:/py_data/final_project/chromedriver_win32/chromedriver.exe')
driver.implicitly_wait(3)
#여성
#driver.get('https://wusinsa.musinsa.com/app/items/lists/001/?category=&d_cat_cd=001&u_cat_cd=&brand=&sort=pop&sub_sort=&display_cnt=90&page=1&page_kind=category&list_kind=small&free_dlv=&ex_soldout=&sale_goods=&exclusive_yn=&price=&color=&a_cat_cd=&sex=28&size=&tag=&popup=&brand_favorite_yn=&goods_favorite_yn=&blf_yn=&campaign_yn=&price1=&price2=&brand_favorite=&goods_favorite=&chk_exclusive=&chk_sale=&chk_soldout=')
driver.get('https://store.musinsa.com/app/items/lists/001001')

In [77]:
link_list = []

In [78]:
try:
    for src in range(3):
        for page in range(4,7):

            ## 'div.listLs_L.buyerL > div > h2 > a' 에 해당하는 elements를 모두 받아온다
            li = driver.find_elements_by_css_selector('div.boxed-list-wrapper > div.sorter-box.box > div > div > a')

            #페이지에 있는 상품 url 저장
            html = driver.page_source
            soup = BeautifulSoup(html, 'html.parser')
            links = soup.select('div.li_inner > div.article_info > p.list_info > a')

            for link in links:
                link_list.append('https://store.musinsa.com'+link['href'])

            ## 다음 페이지 클릭
            driver.find_element_by_xpath('//*[@id="contentsItem_list"]/div[2]/div[1]/div/div/a[%s]' %page).click()
            time.sleep(0.5)
except:
    print("더 이상 페이지가 없습니다.")

In [79]:
link_list = list(set(link_list))

In [80]:
len(link_list)

210

In [81]:
# csv 파일로 저장
import pandas as pd
results = pd.DataFrame(link_list, columns=['link_addr'])
results.to_csv('./link_musinsa_top.csv', header=False) #, mode='a', header=False)

In [82]:
# csv 파일 가져오기
import csv
f = open('./link_musinsa_top.csv','rt')
link_addr_file = csv.reader(f)

In [83]:
link_list = []
for row in link_addr_file :
    link_list.append(row[1])

In [84]:
link_list

['https://store.musinsa.com/app/product/detail/1374391/0',
 'https://store.musinsa.com/app/product/detail/1364418/0',
 'https://store.musinsa.com/app/product/detail/1108007/0',
 'https://store.musinsa.com/app/product/detail/1382538/0',
 'https://store.musinsa.com/app/product/detail/1382540/0',
 'https://store.musinsa.com/app/product/detail/1402491/0',
 'https://store.musinsa.com/app/product/detail/1367104/0',
 'https://store.musinsa.com/app/product/detail/364484/0',
 'https://store.musinsa.com/app/product/detail/86965/0',
 'https://store.musinsa.com/app/product/detail/1402944/0',
 'https://store.musinsa.com/app/product/detail/1377540/0',
 'https://store.musinsa.com/app/product/detail/999580/0',
 'https://store.musinsa.com/app/product/detail/1386089/0',
 'https://store.musinsa.com/app/product/detail/996178/0',
 'https://store.musinsa.com/app/product/detail/1382539/0',
 'https://store.musinsa.com/app/product/detail/1386087/0',
 'https://store.musinsa.com/app/product/detail/746276/0',
 'h

In [85]:
len(link_list)

210

In [86]:
#상세 페이지 들어가서 원하는 정보 저장
i = 0

while i < len(link_list):
    text_list = []
    
    driver.get(link_list[i])
    print(link_list[i])
    
    f = uniform(1.3, 1.6)
    time.sleep(f)
    
    a = ""
    a = link_list[i].split("/detail/")
    a[1][:6]
    
    i += 1
        
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    try:
        images = soup.find('div.product-img')
        name = soup.find('div.right_area.page_detail_product > div.right_contents.section_product_summary > span').get_text()
        price = soup.find('#goods_price').get_text()
        #tag=soup.find('div','_24FKuaBdoK').get_text()
        #info = [soup.find_all('th','_15qeGNn6Dt')[n].string for n in range(0,20)]
        #detail = [soup.find_all('td','jvlKiI0U_y')[n].string for n in range(0,20)]
        #texts = soup.find_all('p', 'se_textarea')


        if not os.path.isdir(a[1][:6]):
            os.mkdir(a[1][:6])     

        #for text in texts:
            #text_list.append(text.get_text().strip().replace(u'\xa0', u' '))
            #text_list2 = [zz for zz in text_list if zz]

        # 긁어올 카테고리 명을 입력하세요     
        data_1 = ({'category':'전체/상의','name': [name], 'price': [price]})
        df1 = pd.DataFrame(data_1)

        #data_2 = [detail]
        #df2 = pd.DataFrame(data_2, columns = info)

        #df3 = pd.concat([df1,df2], axis=1)

        df1.to_csv(".\\"+a[1][:10]+"\\csv_"+a[1][:10]+".csv", index = True, encoding='utf-8')

        for image in images:

            src_page = requests.get(image['src'], headers={"Referer":"https://store.musinsa.com//"})
            src_image = src_page.content
            with open(".\\"+a[1][:10]+"\\img_"+a[1][:10]+'_'+str(images.index(image))+".png", "wb") as downfile:
                downfile.write(src_image)
            f = uniform(0.4, 0.6)    
            time.sleep(f)

    except:
        print("상품정보 가져오기 오류발생")
        pass

https://store.musinsa.com/app/product/detail/1374391/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1364418/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1108007/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1382538/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1382540/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1402491/0
상품정보 가져오기 오류발생
https://store.musinsa.com/app/product/detail/1367104/0
상품정보 가져오기 오류발생


KeyboardInterrupt: 